In [7]:
%matplotlib inline
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import time
import geopandas
from IPython.display import display # display(df) shows dataframe in html formatting
from pandas.tseries.holiday import USFederalHolidayCalendar

print(sys.version)
print(np.__version__)
print(pd.__version__)

3.5.2 |Anaconda custom (64-bit)| (default, Jul  5 2016, 11:41:13) [MSC v.1900 64 bit (AMD64)]
1.11.1
0.18.1


In [8]:
## Example for displaying more of a dataframe
# with pd.option_context('display.max_rows',300,'display.max_columns', 100):
#     display(df)

In [9]:
# default paths for raw and clean data files
data_path = './Fire_Department_Calls_for_Service.csv'
clean_save_path = './clean_sf_fire.csv'

In [10]:
data_types={'Incident Number':np.int64, 'Call Type':str, 'On Scene DtTm':str,
            'Received DtTm': str, 'Zipcode of Incident':np.float64, 'Unit Type':str,
            'Unit sequence in call dispatch':np.float64,'Location':str, 'RowID':str}
date_cols = ['Received DtTm']

In [11]:
df = pd.read_csv(data_path, dtype=data_types, usecols=data_types.keys(), parse_dates=date_cols, na_values='None')

In [12]:
with pd.option_context('display.max_rows',300,'display.max_columns', 100):
    display(df.head(200))

,Incident Number,Call Type,Call Date,Received DtTm,On Scene DtTm,Zipcode of Incident,Unit Type,Unit sequence in call dispatch,Location,RowID
0,306091,Medical Incident,04/12/2000,2000-04-12 21:00:29,NaN,94116.0,ENGINE,1.0,"(37.7487247711275, -122.495504020186)",001030101-E18
1,30612,Medical Incident,04/12/2000,2000-04-12 21:09:02,04/12/2000 09:19:36 PM,94122.0,MEDIC,2.0,"(37.7540326780595, -122.502185504543)",001030104-M14
2,30614,Medical Incident,04/12/2000,2000-04-12 21:09:44,04/12/2000 09:14:11 PM,94102.0,MEDIC,1.0,"(37.7764405100838, -122.418481123408)",001030106-M36
3,30615,Alarms,04/12/2000,2000-04-12 21:13:47,04/12/2000 09:20:12 PM,94102.0,ENGINE,3.0,"(37.7825474000421, -122.412247935495)",001030107-E01
4,30616,Medical Incident,04/12/2000,2000-04-12 21:14:43,04/12/2000 09:20:08 PM,94108.0,RESCUE SQUAD,2.0,"(37.7863072236365, -122.405294845215)",001030108-RS1
5,30620,Citizen Assist / Service Call,04/12/2000,2000-04-12 21:24:27,04/12/2000 09:30:26 PM,94109.0,TRUCK,1.0,"(37.7884388430214, -122.414352343522)",001030112-T03
6,30624,Electrical Hazard,04/12/2000,2000-04-12 21:25:55,04/12/2000 09:31:18 PM,94109.0,ENGINE,1.0,"(37.7903696585646, -122.422328957341)",001030116-E38
7,30626,Odor (Strange / Unknown),04/12/2000,2000-04-12 21:27:55,04/12/2000 09:34:57 PM,94112.0,ENGINE,1.0,"(37.7183153380753, -122.441921085449)",001030117-E15
8,30625,Medical Incident,04/12/2000,2000-04-12 21:27:45,04/12/2000 09:32:34 PM,NaN,ENGINE,1.0,"(37.7750268633971, -122.392346204303)",001030118-E08
9,30628,Medical Incident,04/12/2000,2000-04-12 21:29:54,04/12/2000 09:37:43 PM,94124.0,MEDIC,1.0,"(37.7335924152271, -122.38701801742)",001030119-M17


In [13]:
# total dispatches and unique incidents
print('Dispatches        :', len(df))
print('Unique Incidents  :', df['Incident Number'].nunique())

Dispatches        : 4437244
Unique Incidents  : 1993975


In [14]:
# Removing records without an on-scene time
df = df[~pd.isnull(df['On Scene DtTm'])]

In [15]:
# total dispatches and unique incidents
print('Dispatches        :', len(df))
print('Unique Incidents  :', df['Incident Number'].nunique())

Dispatches        : 3403057
Unique Incidents  : 1877575


In [16]:
# Filtering down to the top three incident call types and medic/private units
call_type_keep = ['Medical Incident', 'Traffic Collision', 'Structure Fire']
unit_type_keep = ['MEDIC','PRIVATE']
df = df[(df['Call Type'].isin(call_type_keep)) & (df['Unit Type'].isin(unit_type_keep))]

In [17]:
# total dispatches and unique incidents
print('Dispatches        :', len(df))
print('Unique Incidents  :', df['Incident Number'].nunique())

Dispatches        : 1345009
Unique Incidents  : 1330410


In [18]:
# Splitting lat/lon field in to two numeric fields
df['lat'], df['lon'] = zip(*df.Location.map(lambda x: [float(val) for val in x.strip('()').split(',')]))

In [53]:
# Getting list of US Federal Holidays (includes observed)
holidays = USFederalHolidayCalendar().holidays(start='2000-01-01', end='2018-01-01')

# Splitting received datetime into separate parts, including flag for weekends
df['year'], df['month'], df['day_of_month'], df['hour_of_day'], df['day_of_year'], df['week_of_year'], df['day_of_week'], df['is_weekend'],  = \
    zip(*df['Received DtTm'].map(lambda val: [val.year, val.month, val.day, val.hour, val.dayofyear, val.week, val.weekday(), val.weekday() in [5,6]]))
    
df['is_holiday'] = df['Received DtTm'].isin(holidays)

In [55]:
with pd.option_context('display.max_rows',300,'display.max_columns', 100):
    display(df.head(200))

,Incident Number,Call Type,Call Date,Received DtTm,On Scene DtTm,Zipcode of Incident,Unit Type,Unit sequence in call dispatch,Location,RowID,lat,lon,year,month,day_of_month,hour_of_day,day_of_year,week_of_year,day_of_week,is_weekend,is_holiday
1,30612,Medical Incident,04/12/2000,2000-04-12 21:09:02,04/12/2000 09:19:36 PM,94122.0,MEDIC,2.0,"(37.7540326780595, -122.502185504543)",001030104-M14,37.754033,-122.502186,2000,4,12,21,103,15,2,False,False
2,30614,Medical Incident,04/12/2000,2000-04-12 21:09:44,04/12/2000 09:14:11 PM,94102.0,MEDIC,1.0,"(37.7764405100838, -122.418481123408)",001030106-M36,37.776441,-122.418481,2000,4,12,21,103,15,2,False,False
9,30628,Medical Incident,04/12/2000,2000-04-12 21:29:54,04/12/2000 09:37:43 PM,94124.0,MEDIC,1.0,"(37.7335924152271, -122.38701801742)",001030119-M17,37.733592,-122.387018,2000,4,12,21,103,15,2,False,False
11,30630,Medical Incident,04/12/2000,2000-04-12 21:31:55,04/12/2000 09:45:22 PM,94122.0,MEDIC,1.0,"(37.763482287794, -122.477678638767)",001030122-M18,37.763482,-122.477679,2000,4,12,21,103,15,2,False,False
12,30633,Medical Incident,04/12/2000,2000-04-12 21:41:20,04/12/2000 09:52:57 PM,94110.0,MEDIC,1.0,"(37.7572324112144, -122.415626198524)",001030124-M09,37.757232,-122.415626,2000,4,12,21,103,15,2,False,False
16,30638,Medical Incident,04/12/2000,2000-04-12 21:49:20,04/12/2000 10:02:05 PM,94116.0,MEDIC,1.0,"(37.7383172638778, -122.499533373462)",001030130-M18,37.738317,-122.499533,2000,4,12,21,103,15,2,False,False
18,30642,Medical Incident,04/12/2000,2000-04-12 21:57:18,04/12/2000 10:05:03 PM,94118.0,MEDIC,1.0,"(37.7781665631084, -122.450200042476)",001030135-M10,37.778167,-122.450200,2000,4,12,21,103,15,2,False,False
21,30647,Medical Incident,04/12/2000,2000-04-12 22:06:10,04/12/2000 10:20:31 PM,94102.0,MEDIC,1.0,"(37.7841909047888, -122.410119483452)",001030139-M41,37.784191,-122.410119,2000,4,12,22,103,15,2,False,False
22,30650,Medical Incident,04/12/2000,2000-04-12 22:08:59,04/12/2000 10:19:49 PM,94115.0,PRIVATE,1.0,"(37.7851959510121, -122.440715841453)",001030142-K439,37.785196,-122.440716,2000,4,12,22,103,15,2,False,False
25,30657,Medical Incident,04/12/2000,2000-04-12 22:29:10,04/12/2000 10:37:28 PM,94117.0,MEDIC,1.0,"(37.7691652248423, -122.453536875377)",001030148-M05,37.769165,-122.453537,2000,4,12,22,103,15,2,False,False


In [56]:
# Loading ZCTA shape file with geopandas
gdf = geopandas.read_file('./sf_zcta.shp')

In [57]:
def get_zcta(point):
    '''
    Takes a geopandas/shapely longitude/latitude point object and returns
    the US Census Zip Code Tabulation Area containing it.
    INPUT: Point must have longitude first, since it expects an x,y coordinate
    OUTPUT: The ZCTA code, or None if not found in San Francisco County
    '''
    zcta = gdf.ZCTA5CE10[gdf.geometry.contains(point)]
    if len(zcta) > 0:
        return zcta.values[0]
    else:
        return None

In [58]:
# Finding ZCTA for each point
df['zcta'] = df.apply(lambda row: get_zcta(geopandas.geoseries.Point(row.lon, row.lat)), axis=1)

In [59]:
df.head()

,Incident Number,Call Type,Call Date,Received DtTm,On Scene DtTm,Zipcode of Incident,Unit Type,Unit sequence in call dispatch,Location,RowID,...,year,month,day_of_month,hour_of_day,day_of_year,week_of_year,day_of_week,is_weekend,is_holiday,zcta
1,30612,Medical Incident,04/12/2000,2000-04-12 21:09:02,04/12/2000 09:19:36 PM,94122.0,MEDIC,2.0,"(37.7540326780595, -122.502185504543)",001030104-M14,...,2000,4,12,21,103,15,2,False,False,94122
2,30614,Medical Incident,04/12/2000,2000-04-12 21:09:44,04/12/2000 09:14:11 PM,94102.0,MEDIC,1.0,"(37.7764405100838, -122.418481123408)",001030106-M36,...,2000,4,12,21,103,15,2,False,False,94102
9,30628,Medical Incident,04/12/2000,2000-04-12 21:29:54,04/12/2000 09:37:43 PM,94124.0,MEDIC,1.0,"(37.7335924152271, -122.38701801742)",001030119-M17,...,2000,4,12,21,103,15,2,False,False,94124
11,30630,Medical Incident,04/12/2000,2000-04-12 21:31:55,04/12/2000 09:45:22 PM,94122.0,MEDIC,1.0,"(37.763482287794, -122.477678638767)",001030122-M18,...,2000,4,12,21,103,15,2,False,False,94122
12,30633,Medical Incident,04/12/2000,2000-04-12 21:41:20,04/12/2000 09:52:57 PM,94110.0,MEDIC,1.0,"(37.7572324112144, -122.415626198524)",001030124-M09,...,2000,4,12,21,103,15,2,False,False,94110


In [60]:
# We need to remove records that are not in a ZCTA. These are likely border cases where a unit was dispatched
# outside of SF for some reason. Most of these cases also have no Zip in the data
print(len(df[pd.isnull(df['zcta'])]))
df = df[~pd.isnull(df['zcta'])]

681


In [61]:
# Setting region to be the ZCTA. This is to keep the region field generic in case we choose to use some other kind
# of region mapping in the future.
df['region'] = df.zcta

### Checking matches

In [62]:
temp = df[~pd.isnull(df['Zipcode of Incident'])].copy()
temp['Zipcode of Incident'] = temp['Zipcode of Incident'].map(lambda x: str(int(x)))
temp['is_match'] = temp['Zipcode of Incident'] == temp['zcta']
len(temp.is_match)

1344176

In [63]:
sum(temp.is_match)

1289950

In [64]:
temp2 = temp[~temp.is_match]
len(temp2)

54226

In [65]:
temp2.head(200)

,Incident Number,Call Type,Call Date,Received DtTm,On Scene DtTm,Zipcode of Incident,Unit Type,Unit sequence in call dispatch,Location,RowID,...,day_of_month,hour_of_day,day_of_year,week_of_year,day_of_week,is_weekend,is_holiday,zcta,region,is_match
27,30668,Medical Incident,04/12/2000,2000-04-12 23:06:10,04/12/2000 11:16:11 PM,94102,MEDIC,1.0,"(37.782943523582, -122.419988441304)",001030159-M03,...,12,23,103,15,2,False,False,94109,94109,False
70,30746,Medical Incident,04/13/2000,2000-04-13 07:20:00,04/13/2000 07:39:10 AM,94122,MEDIC,1.0,"(37.7689370751235, -122.459385072407)",001040058-M12,...,13,7,104,15,3,False,False,94117,94117,False
182,30889,Medical Incident,04/13/2000,2000-04-13 16:06:29,04/13/2000 04:17:12 PM,94112,MEDIC,1.0,"(37.7110980777154, -122.428720598777)",001040208-M15,...,13,16,104,15,3,False,False,94134,94134,False
197,30903,Medical Incident,04/13/2000,2000-04-13 16:41:04,04/13/2000 04:50:05 PM,94102,MEDIC,2.0,"(37.7852197348475, -122.406674658987)",001040223-M41,...,13,16,104,15,3,False,False,94103,94103,False
321,31043,Medical Incident,04/14/2000,2000-04-14 05:54:54,04/14/2000 06:02:34 AM,94131,MEDIC,2.0,"(37.7459562953856, -122.454239907613)",001050046-M12,...,14,5,105,15,4,False,False,94127,94127,False
391,31135,Medical Incident,04/14/2000,2000-04-14 12:00:05,04/14/2000 12:08:49 PM,94114,PRIVATE,1.0,"(37.7696025973262, -122.426309439286)",001050139-A897,...,14,12,105,15,4,False,False,94103,94103,False
427,31196,Medical Incident,04/14/2000,2000-04-14 15:17:10,04/14/2000 03:36:54 PM,94134,MEDIC,1.0,"(37.7131431588266, -122.433706822023)",001050196-M18,...,14,15,105,15,4,False,False,94112,94112,False
458,31228,Medical Incident,04/14/2000,2000-04-14 16:36:44,04/14/2000 04:43:19 PM,94123,MEDIC,2.0,"(37.7944243170715, -122.43410767247)",001050234-M02,...,14,16,105,15,4,False,False,94115,94115,False
489,31262,Medical Incident,04/14/2000,2000-04-14 18:27:25,04/14/2000 06:44:27 PM,94112,MEDIC,1.0,"(37.7174212080814, -122.46263331106)",001050267-M15,...,14,18,105,15,4,False,False,94132,94132,False
583,31380,Medical Incident,04/15/2000,2000-04-15 00:37:11,04/15/2000 01:11:53 AM,94112,MEDIC,1.0,"(37.7314401006954, -122.450004426341)",001060006-M15,...,15,0,106,15,5,True,False,94127,94127,False


Roughly 54K records where the ZCTA does not map the zip code. We should expect cases like this, since the ZCTAs are more regularly shaped and don't perfectly align with zips. I did some spot checking and found only cases like this or cases where the lat/lon lay right on the border between zip codes (according to this tool that maps zips http://maps.huge.info/zip.htm. Note, that tool doesn't do a good job of locating by lat/lon. I found the lat/lon on google maps and then found the corresponding spot on the zip code map to check.) The border cases could be the result of differences in how boarders are treated in shape files, or possibly that the even actually occured within a ZCTA/zip but the nearest intersection was on the border. Either way, I think we should stick with ZCTA mappings. It solves more problems than it creates, and the instances where it doesn't match the zip are few and likely to average out.

In [66]:
# total dispatches and unique incidents
print('Dispatches        :', len(df))
print('Unique Incidents  :', df['Incident Number'].nunique())

Dispatches        : 1344328
Unique Incidents  : 1329765


In [67]:
with pd.option_context('display.max_rows',300,'display.max_columns', 100):
    display(df.head(200))

,Incident Number,Call Type,Call Date,Received DtTm,On Scene DtTm,Zipcode of Incident,Unit Type,Unit sequence in call dispatch,Location,RowID,lat,lon,year,month,day_of_month,hour_of_day,day_of_year,week_of_year,day_of_week,is_weekend,is_holiday,zcta,region
1,30612,Medical Incident,04/12/2000,2000-04-12 21:09:02,04/12/2000 09:19:36 PM,94122.0,MEDIC,2.0,"(37.7540326780595, -122.502185504543)",001030104-M14,37.754033,-122.502186,2000,4,12,21,103,15,2,False,False,94122,94122
2,30614,Medical Incident,04/12/2000,2000-04-12 21:09:44,04/12/2000 09:14:11 PM,94102.0,MEDIC,1.0,"(37.7764405100838, -122.418481123408)",001030106-M36,37.776441,-122.418481,2000,4,12,21,103,15,2,False,False,94102,94102
9,30628,Medical Incident,04/12/2000,2000-04-12 21:29:54,04/12/2000 09:37:43 PM,94124.0,MEDIC,1.0,"(37.7335924152271, -122.38701801742)",001030119-M17,37.733592,-122.387018,2000,4,12,21,103,15,2,False,False,94124,94124
11,30630,Medical Incident,04/12/2000,2000-04-12 21:31:55,04/12/2000 09:45:22 PM,94122.0,MEDIC,1.0,"(37.763482287794, -122.477678638767)",001030122-M18,37.763482,-122.477679,2000,4,12,21,103,15,2,False,False,94122,94122
12,30633,Medical Incident,04/12/2000,2000-04-12 21:41:20,04/12/2000 09:52:57 PM,94110.0,MEDIC,1.0,"(37.7572324112144, -122.415626198524)",001030124-M09,37.757232,-122.415626,2000,4,12,21,103,15,2,False,False,94110,94110
16,30638,Medical Incident,04/12/2000,2000-04-12 21:49:20,04/12/2000 10:02:05 PM,94116.0,MEDIC,1.0,"(37.7383172638778, -122.499533373462)",001030130-M18,37.738317,-122.499533,2000,4,12,21,103,15,2,False,False,94116,94116
18,30642,Medical Incident,04/12/2000,2000-04-12 21:57:18,04/12/2000 10:05:03 PM,94118.0,MEDIC,1.0,"(37.7781665631084, -122.450200042476)",001030135-M10,37.778167,-122.450200,2000,4,12,21,103,15,2,False,False,94118,94118
21,30647,Medical Incident,04/12/2000,2000-04-12 22:06:10,04/12/2000 10:20:31 PM,94102.0,MEDIC,1.0,"(37.7841909047888, -122.410119483452)",001030139-M41,37.784191,-122.410119,2000,4,12,22,103,15,2,False,False,94102,94102
22,30650,Medical Incident,04/12/2000,2000-04-12 22:08:59,04/12/2000 10:19:49 PM,94115.0,PRIVATE,1.0,"(37.7851959510121, -122.440715841453)",001030142-K439,37.785196,-122.440716,2000,4,12,22,103,15,2,False,False,94115,94115
25,30657,Medical Incident,04/12/2000,2000-04-12 22:29:10,04/12/2000 10:37:28 PM,94117.0,MEDIC,1.0,"(37.7691652248423, -122.453536875377)",001030148-M05,37.769165,-122.453537,2000,4,12,22,103,15,2,False,False,94117,94117


## Grouping by day of the year and hour of the day

In [68]:
# filtering for fields that will be retained in training set
keep_fields = ['year', 'month', 'day_of_month', 'hour_of_day', 'day_of_year',
               'week_of_year', 'day_of_week', 'is_weekend', 'is_holiday',
               'region']

df = df[keep_fields]

In [69]:
# creating a unique region and time field for grouping on an hourly basis in each region
df['region_time'] = df.apply(lambda row: '-'.join([str(row.region), str(row.year), str(row.month), str(row.day_of_month), str(row.hour_of_day)]), axis=1)

In [70]:
# Creating dictionary of the number of dispatches in each region/hour tuple
# The 'year' field is arbitrary, I just needed it to return a series of counts instead of a dataframe
dispatch_counts = dict(df.groupby(['region_time'])['year'].count())

In [71]:
# Verifying that count matches
print(sum(dispatch_counts.values()))
print(len(df))

1344328
1344328


In [72]:
# Dropping duplicate region_time rows
df.drop_duplicates(subset='region_time', inplace=True)

In [73]:
# assigning dispatch counts and checking total count again
df['dispatch_count'] = df.apply(lambda row: dispatch_counts[row.region_time], axis=1)

sum(df.dispatch_count)

1344328

In [76]:
len(df)

981987

In [74]:
with pd.option_context('display.max_rows',300,'display.max_columns', 100):
    display(df.head(200))

,year,month,day_of_month,hour_of_day,day_of_year,week_of_year,day_of_week,is_weekend,is_holiday,region,region_time,dispatch_count
1,2000,4,12,21,103,15,2,False,False,94122,94122-2000-4-12-21,2
2,2000,4,12,21,103,15,2,False,False,94102,94102-2000-4-12-21,1
9,2000,4,12,21,103,15,2,False,False,94124,94124-2000-4-12-21,1
12,2000,4,12,21,103,15,2,False,False,94110,94110-2000-4-12-21,1
16,2000,4,12,21,103,15,2,False,False,94116,94116-2000-4-12-21,1
18,2000,4,12,21,103,15,2,False,False,94118,94118-2000-4-12-21,1
21,2000,4,12,22,103,15,2,False,False,94102,94102-2000-4-12-22,1
22,2000,4,12,22,103,15,2,False,False,94115,94115-2000-4-12-22,1
25,2000,4,12,22,103,15,2,False,False,94117,94117-2000-4-12-22,1
26,2000,4,12,22,103,15,2,False,False,94103,94103-2000-4-12-22,2


In [75]:
# df.to_csv('./sf_ems_clean.csv')